In [5]:
import json
import requests
import pandas as pd

In [38]:
response = requests.get('https://www.forbes.com/ajax/list/data?year=2018&uri=billionaires&type=person')
results = response.json()
new_df = pd.DataFrame(results)

In [41]:
new_df.head()

,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,country,...,state,headquarters,position,rank,worth,title,government,pay,managementAssets,salary
0,abdulsamad rabiu,Rabiu,abdulsamad-rabiu,abdulsamad-rabiu,0.000,59.0,"cement, sugar",Diversified,M,Nigeria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adam kwok,Kwok,adam-kwok,no-pic,-10.358,36.0,real estate,Real Estate,M,Hong Kong,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aiyawatt srivaddhanaprabha & family,Srivaddhanaprabha,aiyawatt-srivaddhanaprabha,no-pic,-0.345,34.0,duty-free,Fashion & Retail,M,Thailand,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alceu elias feldmann,Feldmann,alceu-elias-feldmann,no-pic,0.000,70.0,fertilizer,Energy,M,Brazil,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aldo bensadoun,Bensadoun,aldo-bensadoun,aldo-bensadoun,0.000,81.0,shoes,Fashion & Retail,M,Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
new_df.columns

Index(['name', 'lastName', 'uri', 'imageUri', 'worthChange', 'age', 'source',
       'industry', 'gender', 'country', 'timestamp', 'realTimeWorth',
       'realTimeRank', 'realTimePosition', 'squareImage', 'state',
       'headquarters', 'position', 'rank', 'worth', 'title', 'government',
       'pay', 'managementAssets', 'salary'],
      dtype='object')

In [95]:
new_df['worth'].unique()

array([    nan, 112000.,  90000.,  84000.,  72000.,  71000.,  70000.,
        67100.,  60000.,  58500.,  50000.,  48800.,  47500.,  46400.,
        46200.,  46000.,  45300.,  42200.,  40100.,  39000.,  38500.,
        38400.,  34900.,  30300.,  30000.,  29800.,  29600.,  27400.,
        27000.,  25300.,  25000.,  23600.,  23500.,  23000.,  22700.,
        22600.,  22000.,  21900.,  21700.,  21200.,  20900.,  20300.,
        20200.,  20100.,  20000.,  19900.,  19500.,  19100.,  18800.,
        18700.,  18600.,  18500.,  18200.,  18000.,  17900.,  17800.,
        17700.,  17500.,  17400.,  17300.,  16800.,  16700.,  16500.,
        16400.,  16300.,  16000.,  15900.,  15800.,  15600.,  15500.,
        15300.,  15100.,  15000.,  14900.,  14800.,  14700.,  14600.,
        14400.,  14100.,  14000.,  13600.,  13400.,  13000.,  12900.,
        12800.,  12700.,  12600.,  12500.,  12300.,  12200.,  12100.,
        12000.,  11900.,  11800.,  11700.,  11600.,  11500.,  11300.,
        11200.,  110

In [10]:
old_df = pd.read_csv('../data/processed/cleaned_output.csv', sep=';')

In [11]:
old_df.head()

,id,fullName,position,lastName,age,gender,country,image,source,worth,worthChange,realTimePosition,sourceDetails
0,1690,olgun zorlu,2208.0,zorlu,54.0,2,turkey,https://specials-images.forbesimg.com/imageser...,diversified,1.0,0.0,1978.0,diversified
1,2599,zhuo jun,2207.0,zhuo,52.0,1,hong kong,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1978.0,printed circuit boards
2,8536,zhu xingming,2206.0,zhu,51.0,0,china,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1856.0,electrical equipment
3,5887,zhou liangzhang,2205.0,zhou,55.0,0,none,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,2134.0,electrical equipment
4,9216,zhao xiaoqiang,2204.0,zhao,51.0,2,china,https://specials-images.forbesimg.com/imageser...,fashion & retail,1.0,0.0,2134.0,"fashion, entertainment"


In [40]:
def lowercase_feature_1(df,col):
    return df[col].str.lower()

new_df['name'] = lowercase_feature_1(new_df, 'name')

In [28]:
len(old_df)

2260

In [69]:
def extract_data(origin, reference, what_o, what_r, cols, output_df):
    output_df = pd.DataFrame(columns=cols)
    for i, j in zip(origin[what_o], range(len(origin))):
        output_df.loc[j] = reference.loc[reference[what_r].str.contains(i)][cols]

In [87]:
pd.merge(old_df['fullName'], new_df[['name', 'age', 'gender', 'country']], left_on='fullName', right_on='name', how='left')

,fullName,name,age,gender,country
0,olgun zorlu,olgun zorlu,56.0,M,Turkey
1,zhuo jun,zhuo jun,54.0,F,Hong Kong
2,zhu xingming,zhu xingming,52.0,M,China
3,zhou liangzhang,zhou liangzhang,57.0,M,China
4,zhao xiaoqiang,zhao xiaoqiang,52.0,M,China
...,...,...,...,...,...
2260,supaluck umpujh & family,NaN,NaN,NaN,NaN
2261,thomas duff,thomas duff,63.0,M,United States
2262,zakhar smushkin,zakhar smushkin,57.0,M,Russia
2263,zhang zhongneng & family,NaN,NaN,NaN,NaN


In [ ]:
df = df1[df1[col] in lista][col2]

In [59]:
dummy_df = pd.DataFrame()
dummy_df

In [67]:
dummy_df.head()

""


In [79]:
#for i, j in zip(old_df['fullName'], :

age_n = new_df.loc[new_df['name'].isin(old_df['fullName'])][['age', 'country']]

In [85]:
type(age_n)

pandas.core.frame.DataFrame